In [1]:
import requests
import shopify
from bs4 import BeautifulSoup
import logging
import re
from dotenv import load_dotenv
import os


logging.basicConfig(level=logging.ERROR)

shop_url = "https://429eef-90.myshopify.com/"
api_version = '2024-01'
# state = binascii.b2a_hex(os.urandom(15)).decode("utf-8")
# redirect_uri = "http://myapp.com/auth/shopify/callback"
# scopes = ['read_apps', 'write_files', 'read_files', 'write_products', 'read_products', 'read_content', 'write_content', 'write_product_feeds', 'read_product_feeds', 'write_product_listings', 'read_product_listings']

load_dotenv()
access_token = (os.getenv('access_token'))

session = shopify.Session(shop_url, api_version, access_token)
shopify.ShopifyResource.activate_session(session)

In [2]:
def get_search_results(url, max_retries=3):
    """
    Fetches and returns the HTML content of a search results page, handling errors and retrying as needed.

    Args:
        url (str): The URL of the search results page.
        max_retries (int): The maximum number of retries before giving up (default: 3).

    Returns:
        str or None: The HTML content of the page on success, or None on failure.
    """

    logger = logging.getLogger(__name__)
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"

    # Create a request with the custom user agent header
    headers = {"User-Agent": user_agent}

    for retry_num in range(max_retries + 1):
        try:
            with requests.get(url, headers=headers) as response:
                response.raise_for_status()  # Raise an exception if the status code is not 200
                logger.info("Successfully fetched search results page: %s", url)
                return response.text
        except requests.HTTPError as e:
            logger.error(
                "HTTP error fetching search results page: %s (status code: %d)", url, e.response.status_code
            )
             

            if retry_num < max_retries:
                logger.info("Retrying request for search results page: %s", url)
                continue  # Retry the request
            else:
                return None
                raise  # Re-raise the exception after all retries are exhausted
        except requests.ConnectionError as e:
            logger.error("Connection error fetching search results page: %s", url)
            if retry_num < max_retries:
                logger.info("Retrying request for search results page: %s", url)
                continue
            else:
                raise  # Re-raise the exception after all retries are exhausted
    return None  # If all retries fail, return None



In [3]:
def sear_Nprod(title):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.

    Gel Polish Deluxe Series D114 (dark, gray-lilac, enamel), 8 ml

    """
    # title = "Gel Polish Deluxe Series D119 (dark, gray-lilac, enamel), 8 ml"

    print('Search')

    name4sear = re.sub(r"\[.*?\]|\(.*?\), ", "", title).lower().replace(" 8ml","").replace("gel polish ","").replace("cat’s","cat").replace(" ","+")
    print(name4sear)
    url = f"https://newbeautylab.shop/?product_cat=&s={name4sear}"

    print(url)
    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        entry_title = soup.find('h2', class_='entry-title')

        # Check if element is found
        if entry_title:
            # Find the a element within the h2
            link = entry_title.find('a')
            
            # Check if link is found
            if link:
                # Extract and return the href attribute
                href = link.get('href')

                html = get_search_results(href)
                if html:
                    soup = BeautifulSoup(html, 'html.parser')
                    links = soup.find_all('span',class_='sku')
                    if links:
                        barcode = links[0].text
                        print(barcode)
                        return barcode
    print ('No barcode')
    return None


In [4]:
class Product:
    def __init__(self, name, spu, imgs, url, detail=None):
        self.name = name
        self.detail = ""
        if detail:
            self.detail = detail
        self.spu = spu
        self.imgs = imgs
        self.url = url
        if '8ml' in name:
            self.price = '9.00'
            self.compare_at_price = '12.00'
            self.grams = 12
            self.weight = 0.012
            self.weight = 0.012
            self.barcode = None
        elif '15' in name:
            self.barcode = None
            self.price = '15.00'
            self.compare_at_price = '17.00'
            self.grams = 20
            self.weight = 0.020
        elif '30' in name:
            self.barcode = None
            self.price = '25.00'
            self.compare_at_price = '30.00'
            self.grams = 50
            self.weight = 0.050
        self.vendor = 'Komilfo'
        self.product_type = 'Gel Polish'

    def __str__(self):
        return f"Product: {self.name}, SPU: {self.spu}"

In [5]:
def extract_product_links(url):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.
    """
    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('h4')
        return [h3.find('a')['href'] for h3 in links if h3.find('a')]
    return None


In [6]:
def UkRefSearch(sku):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.
    """
    url = 'https://reformacosmetics.com/search/?q='+str(sku)+'&s=%D0%9F%D0%BE%D1%88%D1%83%D0%BA'

    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', class_='nm')
        return [link['href'] for link in links]
    return None


In [7]:
def letsgo(url):
    print()

    # url = 'https://reforma.top/catalogsearch/result/?q=' + SPU + '&product_list_limit=144'
    # url = 'https://reforma.top/shop/nails/gel-polish/base-tops/?product_list_limit=144&p='+str(page)
    print(url)
    product_links = extract_product_links(url)
    if product_links:
        print("List created")
        return product_links
        

In [8]:
def extract_data_from_url(url):
    """
    Extracts product data from the provided URL.
    """
    try:
        # Extract data from the URL
        html = get_search_results(url)
        if not html:
            raise ValueError("Failed to fetch HTML content from the URL.")

        soup = BeautifulSoup(html, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            del a_tag['href']        
        description_div = soup.find('div', id='tab-description')

        # Remove h3 and li elements with data attribute 'data-number-of-phrases="9"'
        if description_div:
            for element in description_div.find_all(['h3', 'li']):
                if 'data-number-of-phrases="9"' in str(element):
                    element.extract()

        # Convert the remaining div element to a string
        detail = str(description_div) if description_div else ""
        spu = soup.find('span', class_='sku').text
        name = soup.find('h1',class_='product_title entry-title').text
        title = name.replace("Komilfo ", "").replace("ML","ml").replace(" ml","ml").replace("Liquid Glam Gel","Gel Polish Liquid Glam Gel")
        imgs = [img.get('data-large_image') for img in soup.find_all('img',decoding="async")]
        # imgs =soup
        
        return Product(title, spu, imgs, url, detail=detail)
    except Exception as e:
        logging.error(f"Failed to extract data from URL: {str(e)}")
        return None

In [9]:
def update_product(shopify_product, barcode):
    """
    Updates an existing Shopify product with new title and handle if needed.
    """

    # print(shopify_product.title)

    variant = shopify_product.variants[0]

    variant.barcode = barcode
    variant.save()
    shopify_product.save()

In [10]:
def create_new_product(product, new_handle):
    """
    Creates a new Shopify product with details and default variant.
    """

    shopify_product = shopify.Product.create({
        'title': product.name,
        'handle': new_handle,
        'body_html': product.detail,
        'vendor': product.vendor,
        'product_type': product.product_type,
        'Status': 'draft'
    })

    default_variant = {
        'title': 'Default Title',
        'price': product.price,
        'sku': product.spu,
        'compare_at_price': product.compare_at_price,
        'inventory_policy': 'deny',
        'fulfillment_service': 'manual',
        'inventory_management': 'shopify',
        'option1': 'Default Title',
        'barcode': product.barcode,
        'taxable': True,
        'grams': product.grams,
        'weight': product.weight,
        'weight_unit': 'kg',
        'requires_shipping': True
    }

    variant = shopify.Variant(default_variant)
    shopify_product.variants = [variant]

    half_len = len(product.imgs) // 2
    
    shopify_product.images = [{'src': img_url, 'position': index, 'alt': product.name} for index, img_url in enumerate(product.imgs[:half_len], start=1)]
    shopify_product.save()
    return shopify_product

In [11]:
def create_product(product):
    """
    Creates a new product on Shopify with the provided data, 
    handling creation, updates, and errors.
    """
    try:
        Handle = re.sub(r"\[.*?\]|\(.*?\), ", "", product.name).lower().replace(" ","-")
        shopify_product = shopify.Product.find(handle=Handle)

        if not shopify_product:
            shopify_product = create_new_product(product, Handle)
            logging.info(f"Product created successfully! (ID: {shopify_product.id})")
            print("Product created successfully!")
        else:
            # Update existing product if titles differ
            shopify_product = shopify_product[0]
            print(shopify_product.title)
            print('Already exist!')
            if product.barcode:
                update_product(shopify_product, product.barcode)
                logging.info(f"Product updated successfully! (ID: {shopify_product.id})")
                # print(shopify_product.title)
                print("Product barcode added successfully!")
            else:
                logging.info(f"Product already exists with matching title: {shopify_product.title} (ID: {shopify_product.id})")
                # print(shopify_product.title)
                print("Barcode not exists!")

        return shopify_product

    except Exception as e:
        logging.error(f"Failed to create product: {str(e)}")
        return None

In [12]:
def create_or_get_collection(col_handle, col_name):
    # Find the collection by handle
    collections = shopify.CustomCollection.find(handle=col_handle)

    # If the collection doesn't exist, create a new one
    if not collections:
        collection = shopify.CustomCollection.create({
            'title':col_name,
            'handle':col_handle}
        )
        print(f"Collection {col_name} created")
        print()
    else:
        # Get the first collection from the PaginatedCollection
        collection = collections[0]
        print(f"Collection {col_name} exists")
    return collection


def add_product_to_collection(collection, product):
    new_collect = shopify.Collect.create({
        "collection_id": collection.id,
        "product_id": product.id
    })
    print(f"Product added to collection")


In [14]:
extract_data_from_url('https://komilfo.ua/en/product/gel-polish-komilfo-kaleidoscopic-collection-k001-light-green-neon-8-ml/').detail

'<div aria-labelledby="tab-title-description" class="woocommerce-Tabs-panel woocommerce-Tabs-panel--description panel entry-content wc-tab" id="tab-description" role="tabpanel">\n<h2>Description</h2>\n<h2><span class="VIiyi" lang="en"><span class="JLqJ4b ChMk0b" data-language-for-alternatives="en" data-language-to-translate-into="ru" data-number-of-phrases="1" data-phrase-index="0">The advantage of choosing Komilfo products:</span></span></h2>\n<ul>\n<li><strong>The new series of the updated Komilfo palette</strong> is high-quality gel polishes at an affordable price, developed by masters for masters!</li>\n<li><strong>Komilfo Kaleidoscopic Collection I – the</strong>\xa0first part of the collection is 10 incredible neon shades that do not need a white backing! The incomparable dense bright pigment, perfect application and stunning rich shades will create exactly that bright unforgettable mood that we need so much.</li>\n<li>Create your own vivid kaleidoscope of events with the new\xa0

In [13]:
def allProdCol(colURL):
    # Get the collection name
    col_handle = colURL.split('/')[-2].replace('-en', '')
    # col_name = col_handle.replace('-', ' ').upper()
    

    # Create or get the collection
    # create_or_get_collection(col_handle, col_name)

    # Loop through pages
    for i in range(1, 35):    
        url = f"{colURL}/page/{i}"

        # Extract product links from the page
        prod_links = extract_product_links(url)

        if prod_links:
            print(f'{col_handle}: Page {i}: {len(prod_links)} products')
            print()
            
            # Add each product to the collection
            for index, prod_link in enumerate(prod_links, start=1):
                data = extract_data_from_url(prod_link)
                create_product(data)
                print(f"{index}/{len(prod_links)}\n")
        else:
            print(f'{col_handle}: Page {i}: No products')
            break

listCol = [
        
           
'https://komilfo.ua/en/product-category/komilfo-base-and-top-coat/komilfo-base-coat/attribute-base-coat-color-base-french-en/attribute-scope-15-ml-en/'
           
           



           
        
           ]

for colURL in listCol:

    allProdCol(colURL)

attribute-scope-15-ml: Page 1: 12 products

Product created successfully!
1/12



KeyboardInterrupt: 

In [ ]:
prod_links = [
    'https://komilfo.ua/en/product/komilfo-glitter-top-top-for-gel-polish-with-glitter-without-sticky-layer-15-ml/'
    

    
]
for prod_link in prod_links:
    data = extract_data_from_url(prod_link)

    create_product(data)

Product created successfully!


In [ ]:
extract_product_links('https://komilfo.ua/en/product-category/komilfo-gel-polish/')

In [ ]:
extract_product_links('https://komilfo.ua/ru/product-category/komilfo-gel-polish-ru/french-collection/')

In [ ]:
Collection.title = "My Collection"
Collection.save()

In [ ]:
# col_name = 'fuch yoo ooo'
# col_handle = 'fuch yoo ooo'
# collection = shopify.SmartCollection.create({
#     'title': col_name,
#     'handle': col_handle,
#     'rules': [
#         {
#             "column": "title",
#             "relation": "contains",
#             "condition": col_name.split(' ')[0]
#         },
#         {
#             "column": "vendor",
#             "relation": "equals",
#             "condition": 'Komilfo'
#         }
#     ]
# })
